Issues with MADE dataset:
FTIR
* Dodgy sample numbers
* 2 Hit confidence columns
* 2 substance detected columns

In [1]:
# Module imports
import copy
import datetime
import os
import numpy as np
import pandas as pd
from collections import namedtuple

def fix_sample_number(x):
    """Make sure all samples numbers are of form: AXXX (where A is one of A, F, W and X is a digit)"""
    if isinstance(x, float) and np.isnan(x):
        return x # leave NaN's alone    
    try:
        sn = int(x)
        sn = 'F{:04d}'.format(int(x))
    except ValueError:
        # Assume string so make sure it's of the right format
        sn = str(x).capitalize()
    assert sn[0] in ['A', 'F', 'W'], "Bad ID %s" % sn
    return sn

def now():
    return datetime.datetime.now().strftime("%d/%m/%y %H:%M:%S")

In [2]:
ftir_csv = 'MADE/FTIR Analysis Data Recording Form.csv'
catalog_csv = 'MADE/Sample Cataloguing Form.csv'
reagent_csv = 'MADE/Reagent Outcomes.csv'
hr_csv = 'MADE/MADE MAST Intervention Questionnaire.csv'

date_cols = ['Timestamp']
df_ftir = pd.read_csv(ftir_csv, engine="python", parse_dates=date_cols)
df_catalog = pd.read_csv(catalog_csv, engine="python", parse_dates=date_cols)
df_reagent = pd.read_csv(reagent_csv, engine="python", parse_dates=date_cols)
df_hr = pd.read_csv(hr_csv, engine="python", parse_dates=date_cols)

mla_excel = 'MADE/MADE - Loop 2018 event results sheet_.xlsx'
df_mla = pd.read_excel(mla_excel, sheetname='MLA', header=1)

# Sort out column names
df_reagent.rename(columns={'Sample Code':'Sample Number', 'Substance(s) detected' : 'Reagent Result'}, inplace=True)
df_hr.rename(columns={'Sample Number:':'Sample Number'}, inplace=True)
df_mla.rename(columns={'Sample Num':'Sample Number'}, inplace=True)

# Make all sample numbers a 4-digit code starting with F
df_ftir['Sample Number'] = df_ftir['Sample Number'].apply(fix_sample_number)
df_catalog['Sample Number'] = df_catalog['Sample Number'].apply(fix_sample_number)
df_reagent['Sample Number'] = df_reagent['Sample Number'].apply(fix_sample_number)
df_hr['Sample Number'] = df_hr['Sample Number'].apply(fix_sample_number)
df_mla['Sample Number'] = df_mla['Sample Number'].apply(fix_sample_number)

# Prune down MLA to valid sample numbers
df_mla = df_mla[df_mla['Sample Number'].notnull()]

DataFrames = namedtuple('DataFrames', ['catalog', 'ftir', 'reagent','mla', 'hr'])
dfs = DataFrames(
    catalog=df_catalog,
    ftir=df_ftir,
    reagent=df_reagent,
    mla=df_mla,
    hr=df_hr)

In [6]:
def gsheets_service():
    from googleapiclient.discovery import build
    from httplib2 import Http
    from oauth2client import file, client, tools
    # If modifying these scopes, delete the file token.json.
    CREDS_FILE = '/opt/random/MADE/JensDataExportJupyter_client_secret.json'
    SCOPES = 'https://www.googleapis.com/auth/spreadsheets.readonly'
    store = file.Storage('token.json')
    creds = store.get()
    if not creds or creds.invalid:
        import argparse
        flags = argparse.ArgumentParser(parents=[tools.argparser]).parse_args([])
        flow = client.flow_from_clientsecrets(CREDS_FILE, SCOPES)
        creds = tools.run_flow(flow, store, flags)
    service = build('sheets', 'v4', http=creds.authorize(Http()))
    return service

# The ID and range of a sample spreadsheet.
BOOMTOWN2018_SPREADSHEET_ID = '1RiA-FwG_954Ger2VPsOSA3JLh-7sEoTYr40eVS0mp24'
MADE2018_SPREADSHEET_ID = '1daXdyL6uL8qnMsEsP0RLZE9nDzt6J7Zr1ygQdguvi-E'
SPREADSHEET_ID = BOOMTOWN2018_SPREADSHEET_ID
CATALOG_RANGE = 'Catalog!A:R'


if True:
    # Call the Sheets API
    result = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET_ID,
                                                range=CATALOG_RANGE).execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')
    else:
        print('Name, Major:')
        for row in values:
            # Print columns A and E, which correspond to indices 0 and 4.
            print('%s, %s' % (row[0], row[4]))




Name, Major:
, Sample Advertised/Acquired/Sold As
Thu 9/08 - 12:19, MDMA
Thu 9/08 - 12:19, Ketamine
Thu 9/08 - 12:23, MDMA
Thu 9/08 - 12:27, MDMA
Thu 9/08 - 12:28, MDMA
Thu 9/08 - 12:29, Ketamine
Thu 9/08 - 12:29, Ketamine
Thu 9/08 - 12:30, Ketamine
Thu 9/08 - 12:31, MDMA
Thu 9/08 - 12:31, MDMA
Thu 9/08 - 12:32, MDMA
Thu 9/08 - 12:32, MDMA
Thu 9/08 - 12:32, MDMA
Thu 9/08 - 12:33, MDMA
Thu 9/08 - 12:34, 2C-B
Thu 9/08 - 12:34, Found or otherwise not known
Thu 9/08 - 12:36, MDMA
Thu 9/08 - 12:36, MDMA
Thu 9/08 - 12:42, MDMA
Thu 9/08 - 12:43, MDMA
Thu 9/08 - 12:44, Ketamine
Thu 9/08 - 12:45, Ketamine
Thu 9/08 - 12:45, Ketamine
Thu 9/08 - 12:46, MDMA
Thu 9/08 - 12:46, Cocaine
Thu 9/08 - 12:47, MDMA
Thu 9/08 - 12:48, MDMA
Thu 9/08 - 12:49, MDMA
Thu 9/08 - 12:51, MDMA
Thu 9/08 - 12:51, MDMA
Thu 9/08 - 12:54, MDMA
Thu 9/08 - 12:56, MDMA
Thu 9/08 - 12:56, MDMA
Thu 9/08 - 12:58, Found or otherwise not known
Thu 9/08 - 12:58, 2C-B
Thu 9/08 - 12:59, MDMA
Thu 9/08 - 13:00, MDMA
Thu 9/08 - 13:00, Ke

Sat 11/08 - 12:27, Found or otherwise not known
Sat 11/08 - 12:27, Found or otherwise not known
Sat 11/08 - 12:27, MDMA
Sat 11/08 - 12:29, MDMA
Sat 11/08 - 12:29, Ketamine
Sat 11/08 - 12:30, MDMA
Sat 11/08 - 12:30, Found or otherwise not known
Sat 11/08 - 12:30, MDMA
Sat 11/08 - 12:32, MDMA
Sat 11/08 - 12:32, LSD
Sat 11/08 - 12:33, Found or otherwise not known
Sat 11/08 - 12:33, MDMA
Sat 11/08 - 12:34, Found or otherwise not known
Sat 11/08 - 12:36, MDMA
Sat 11/08 - 12:37, Cocaine
Sat 11/08 - 12:38, MDMA
Sat 11/08 - 12:38, Found or otherwise not known
Sat 11/08 - 12:39, MDMA
Sat 11/08 - 12:40, Ketamine
Sat 11/08 - 12:40, Ketamine
Sat 11/08 - 12:41, MDMA
Sat 11/08 - 12:42, MDMA
Sat 11/08 - 12:42, MDMA
Sat 11/08 - 12:43, Found or otherwise not known
Sat 11/08 - 12:44, Cocaine
Sat 11/08 - 12:45, Cocaine
Sat 11/08 - 12:45, MDMA
Sat 11/08 - 12:45, MDMA
Sat 11/08 - 12:46, Found or otherwise not known
Sat 11/08 - 12:47, Ketamine
Sat 11/08 - 12:49, MDMA
Sat 11/08 - 12:51, MDMA
Sat 11/08 - 12:5

In [4]:
# Check for duplicates
if dfs.catalog['Sample Number'].duplicated().any():
    print("### Duplicated catalog sample numbers ###")
    print(dfs.catalog[dfs.catalog['Sample Number'].duplicated(keep=False)])
if dfs.ftir['Sample Number'].duplicated().any():
    print("### Duplicated FTIR sample numbers ###")
    print(dfs.ftir[dfs.ftir['Sample Number'].duplicated(keep=False)])
if dfs.reagent['Sample Number'].duplicated().any():
    print("### Duplicated catalog sample numbers ###")
    print(dfs.reagent[dfs.reagent['Sample Number'].duplicated(keep=False)])
if dfs.hr['Sample Number'].duplicated().any():
    print("### Duplicated catalog sample numbers ###")
    print(dfs.hr[dfs.hr['Sample Number'].duplicated(keep=False)])
if dfs.mla['Sample Number'].duplicated().any():
    print("### Duplicated MLA sample numbers ###")
    print(dfs.mla[dfs.mla['Sample Number'].duplicated(keep=False)])
    
    
if dfs.catalog['Sample Number'].duplicated().any() or \
    dfs.ftir['Sample Number'].duplicated().any() or \
    dfs.reagent['Sample Number'].duplicated().any() or \
    dfs.hr['Sample Number'].duplicated().any() or \
    dfs.mla['Sample Number'].duplicated().any():

    outs = 'Please fix duplicated values'
    print(outs)
#     raise RuntimeError(outs)

### Duplicated catalog sample numbers ###
             Timestamp Sample Number Your initials Sample Source  \
22 2018-07-28 14:34:58         F0023            JF        Public   
27 2018-07-28 15:10:42         F0023            JF        Public   

   Sample Advertised/Acquired/Sold As  \
22                               MDMA   
27                               MDMA   

   Has the Service User or a close friend tried this batch? User Suspicion  \
22                                                 No                  NaN   
27                                                 No                  NaN   

             Sample Form What is the logo?  \
22  Partial ecstasy pill               n/a   
27  Partial ecstasy pill               NaN   

   Which device was a photo taken with? Who does it belong to?  \
22   N/A: fragment too small, doesn't warrant a photo            
27                                                NaN            

    What is the mass? (mg) What is the shape of the pill

In [5]:
# Check there are no sample numbers in any of the other spreadsheets that aren't in the cataolog sheet
catalog_unique = set(dfs.catalog['Sample Number'].unique())

ftir_unique = set(dfs.ftir['Sample Number'].unique())
ftir_orphan = ftir_unique.difference(catalog_unique)
if ftir_orphan:
    print("Orphaned FTIR sample numbers: %s" % sorted(ftir_orphan))

reagent_unique = set(dfs.reagent['Sample Number'].unique())
reagent_orphan = reagent_unique.difference(catalog_unique)
if reagent_orphan:
    print("Orphaned Reagent Test sample numbers: %s" % sorted(reagent_orphan))

hr_unique = set(dfs.hr['Sample Number'].unique())
hr_orphan = hr_unique.difference(catalog_unique)
if hr_orphan:
    print("Orphaned HR sample numbers: %s" % sorted(hr_orphan))
    
mla_unique = set(dfs.mla['Sample Number'].unique()).difference(catalog_unique)
mla_orphan = mla_unique.difference(catalog_unique)
if mla_orphan:
    print("Orphaned MLA sample numbers: %s" % sorted(mla_orphan))
    
# Check for any that are only in the catalog
outside_catalog = set.union(reagent_unique, hr_unique, mla_unique)
catalog_only = catalog_unique.difference(outside_catalog)
if catalog_only:
    print("Sample numbers only in catalog: %s" % sorted(catalog_only))
    
# Check for any that aren't in FTIR and don't have anything in reagent test
ftir_missing = catalog_unique.difference(ftir_unique).difference(reagent_unique).difference(catalog_only)
if len(ftir_missing):
    print("Samples not in FTIR or Reagent: %s" % sorted(ftir_missing))

all_unique = copy.copy(ftir_unique)
all_unique.update(reagent_unique, hr_unique, mla_unique)
if (all_unique or catalog_only):
    outs = "### Please fix orphaned/catalog only samples ###"
    print(outs)
    #raise RuntimeError(outs)

Orphaned FTIR sample numbers: ['F0547']
Orphaned HR sample numbers: ['F0000']
Sample numbers only in catalog: ['A0051', 'A0052', 'A0053', 'A0054', 'A0056', 'F0006', 'F0011', 'F0015', 'F0028', 'F0036', 'F0041', 'F0049', 'F0060']
Samples not in FTIR or Reagent: ['F0004', 'F0010', 'F0014', 'F0033', 'F0042', 'F0072']
### Please fix orphaned/catalog only samples ###


In [6]:
# Clean up catalog
# Drop all unwanted columns
l = ['Your initials',
     'Which device was a photo taken with? Who does it belong to?',
     'Is a breakline present?',
     'Unusual appearance'
    ]
dfs.catalog.drop(l, axis=1, inplace=True)

d = {
    'Timestamp' : 'Catalog timestamp',
    'Sample Advertised/Acquired/Sold As': 'Catalog Sold As',
    'Sample Form' : 'Catalog form',
    'Has the Service User or a close friend tried this batch?': 'Catalog tried',
    'What is the mass? (mg)': 'Full pill mass',
    'What is the shape of the pill?': 'Pill shape',
    'What is the logo?': 'Pill logo',
    'What colour is the pill?': 'Pill colour'
}
dfs.catalog.rename(columns=d, inplace=True)

In [7]:
# For FTIR columns need to merge the data from the 'Compound detected', 'Hit Confidence.1' columns into the
# 'Substance detected', 'Hit Confidence' column where the substance detected was 'other'
mask = dfs.ftir['Substance detected'] != 'Other'
dfs.ftir['Substance detected'].where(mask, dfs.ftir['Compound detected'], inplace=True) # Copy values from 'Compound detected'
dfs.ftir['Hit Confidence'].where(mask, dfs.ftir['Hit Confidence.1'], inplace=True)
dfs.ftir.drop(['Compound detected', 'Hit Confidence.1', 'Brief Note'], axis=1, inplace=True)

mask = dfs.ftir['Compound detected (Subtraction)'] != 'Other'
dfs.ftir['Compound detected (Subtraction)'].where(mask, df_ftir['Substance detected.1'], inplace=True) # Copy values from 'Compound detected'
dfs.ftir['Hit Confidence.2'].where(mask, dfs.ftir['Hit Confidence.3'], inplace=True)
dfs.ftir.drop(['Substance detected.1', 'Hit Confidence.3', 'Brief Note.1'], axis=1, inplace=True)

# Drop all unwanted columns
l = ['Your name and surname initial',
     'User Suspicion',
     'Is anything detected after subtraction analysis?',
     'Analysis required',
     'Note for harm reduction worker'
    ]
dfs.ftir.drop(l, axis=1, inplace=True)

# Rename shared columns so that we can check for any errors and remove any columns not of interest to the master df
d = {
    'Timestamp' : 'FTIR timestamp',
    'Sample Sold As': 'FTIR Sold As',
    'Sample Form' : 'FTIR form',
    'Has the Service User or a close friend tried this batch?': 'FTIR tried',
    'Substance(s) detected' : 'FTIR final result',
    'Substance detected' : 'FTIR result1',
    'Hit Confidence' :  'FTIR hit1',
    'Is anything detected after subtraction analysis?' : 'FTIR subtraction positive',
    'Compound detected (Subtraction)' :  'FTIR result2',
    'Hit Confidence.2' :  'FTIR hit2',
    '"Strength" of powdered substance' : 'FTIR Powder Strength',
    'Does the substance detected match the substance that was advertised?' : 'FTIR Matches Sold As',
}
dfs.ftir.rename(columns=d, inplace=True)

In [8]:
# Clean up HR form

# Drop all unwanted columns
l = ['HR worker name:']
dfs.hr.drop(l, axis=1, inplace=True)

# Rename shared columns so that we can check for any errors and remove any columns not of interest to the master df
d = {
    'Timestamp' : 'HR timestamp',
    'You submitted a substance for analysis. What were you told it was when you got it?': 'HR Sold as',
    'Had you already tried this substance before getting it tested?': 'HR tried',
    'What was your first sample number at this event? Did you take a photo or keep the ticket?': 'Previous Sample Number'
}
dfs.hr.rename(columns=d, inplace=True)

In [9]:
# Catalog and FTIR data frames
df_all = pd.merge(dfs.catalog, dfs.ftir, how='left', on=['Sample Number'])

In [10]:
# Merge in any reagent test data
df_all = pd.merge(df_all, dfs.reagent[['Sample Number', 'Reagent Result']], how='left', on=['Sample Number'])

In [11]:
# Merge in any pill strength data
df_all = pd.merge(df_all, dfs.mla[['Sample Number', 'MDMA / tablet (mg)', '% MDMA content']], how='left', on=['Sample Number'])

In [12]:
# Merge in HR data
df_all = pd.merge(df_all, dfs.hr, how='left', on=['Sample Number'])

In [13]:
# Fix column orders
prefix = ['Sample Number',
          'Catalog timestamp', 'FTIR timestamp', 'HR timestamp',
          'Catalog Sold As', 'FTIR Sold As','HR Sold as', 
          'Catalog form', 'FTIR form',
          'Catalog tried', 'FTIR tried', 'HR tried']
columns = [c for c in df_all.columns if c not in prefix]
columns = prefix + columns
df_all = df_all[columns]
df_all.to_csv('foo.csv')